In [3]:
import numpy as np

#Adaboost分类
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier


def calculateAlpha(error):
    if error == 0:
        return 0
    else:
        return (1/2) * (np.log((1-error)/error))

def calculateWeight(alpha, originalWeight, y, g):
    weight = originalWeight * (np.e ** (-alpha * y * g))
    return weight

def sign(value):
    if value > 0:
        return 1
    else:
        return -1

def learn(x, alpha, featureValue):
    if x >= featureValue:
        return sign(alpha * 1)
    else:
        return sign(alpha * -1)

#参数是整个特征集，目标划分，对应的权重
def findBestCut(dataset, target, weights):
    shape = np.shape(dataset)
    featureNo = shape[1]
    newWeights = []
    for featureSeq in range(featureNo):
        featureRow = dataset[:, featureSeq]
        for featureIndex in range(len(featureRow)):
            featureValue = featureRow[featureIndex]
            resultRow = featureRow.copy()
            for index in range(len(resultRow)):
                if resultRow[index] >= featureValue:
                    resultRow[index] = 1
                else:
                    resultRow[index] = -1

            #得到的result和target做对比，不一样则划分错误，乘以相应的权值作为error
            error = np.dot(np.multiply(resultRow, target), weights.T)
           #如果error小于0.5，说明这是个合格的弱分类器G，记录下相应的弱分类器权重W和划分值以及划分值所在的特征组并返回
            if error < 0.5:
                alpha = calculateAlpha(error)
                print('alpha is :' + str(alpha))
                for weightNo in range(len(weights)):
                    g = learn(featureRow[weightNo], alpha, featureValue)
                    newWeights.append(calculateWeight(alpha, weights[weightNo], target[weightNo], g))

                newWeights = newWeights / sum(newWeights)
                
                return featureValue, alpha, newWeights
    return None, None, newWeights

    # newWeights = newWeights/sum(newWeights)
    #
    # return featureValue, alpha, newWeights


#循环生成弱分类器
def adaBoost(dataset,target,weekClassifierNo=4):
    shape = np.shape(dataset)
    recordNo = shape[0]
    weights = np.ones(recordNo)/recordNo
    featureNo = shape[1]
    weekClassifiers = {}
    for classifierSeq in range(weekClassifierNo):
        featureValue, alpha, newWeights = findBestCut(dataset, target, weights)
        if len(newWeights) == 0:
            break
        weekClassifiers["WClassifier" + str(classifierSeq)] = [featureNo, featureValue, alpha]
        weights = newWeights
    return weekClassifiers

iris = load_iris()
#简单处理数据，分成种类1和-1
iris.target[iris.target > 0] = 1
iris.target[iris.target == 0] = -1
weekClassifiers = adaBoost(iris.data, iris.target, 50)
print(weekClassifiers)

alpha is :0.09360577104407318
alpha is :0.03509886861946332
alpha is :0.09009038757777348
alpha is :0.2430797244206212
alpha is :0.17190165288528267
alpha is :0.5145727335107108
alpha is :nan
{'WClassifier0': [4, 4.7, 0.09360577104407318], 'WClassifier1': [4, 4.9, 0.03509886861946332], 'WClassifier2': [4, 4.9, 0.09009038757777348], 'WClassifier3': [4, 4.9, 0.2430797244206212], 'WClassifier4': [4, 5.1, 0.17190165288528267], 'WClassifier5': [4, 5.1, 0.5145727335107108], 'WClassifier6': [4, 5.1, nan]}


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in log
  del sys.path[0]
